In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.sparse import spdiags
from scipy.sparse.linalg import spsolve
from scipy.sparse.linalg import eigsh
from scipy.sparse import identity as sid
%matplotlib inline

# DUE 12/18/19, by 5:00 PM

**Problem 1**: (20pts) In quantum mechanics, it is really common to see boundary value problems of the form 

$$
-\epsilon\frac{d^{2}y}{dx^2} + \cos(\pi x)y = Ey, ~ y(-1) = y(1).
$$

where $0\leq \epsilon \ll 1$, i.e. we let $\epsilon$ be a small positive parameter.  $E\geq 0$ is the _energy_ of a particle trapped in an oscillating potential well $V(x) = \cos(\pi x)$, which is formed in crystal lattices of metals.  We likewise use _periodic-boundary conditions_ by setting 

$$
y(-1) = y(1).
$$

We desribe the probability of a particle being in the interval $[-1,a]$, $a<1$ via the formula

$$
P(-1\leq x \leq a) = \int_{-1}^{a} \tilde{y}(x), ~ \tilde{y}(x) = \frac{y^{2}(x)}{\int_{-1}^{1}y^{2}(x)dx}
$$

1a) Using second-order centered-difference approximations and spdiags, write code which discretizes the operator $-\epsilon\frac{d^{2}y}{dx^2} + \cos(\pi x)y\approx A{\bf y}$, where ${\bf y}=\left(y_{1} ~y_{2}\cdots y_{N-1}\right)^{T}$, $y_{j}=y(x_{j})$.  Note, the periodic boundary conditions are implemented as 

$$
y_{0} = y_{N-1}, ~ y_{N} = y_{1}.
$$

1b) You have now formed a discrete eigenvalue problem $A{\bf y} = E{\bf y}$.  Using the code below find the first 10 eigenvalues of the discretized equations.  For $\epsilon=1,.1,.01$ and $.001$, describe via a well designed plot how the first ten energy levels change as you decrease $\epsilon$.  

1c) For $\epsilon=.01$, compare the associated probability distributions $\tilde{y}(x)$ for the first three energy levels.  How do the likelihoods of where a particle would be found change with changing energy?   

In [ ]:
def eval_find(Nvls,xvals,epvl):
    Nint = int(Nvls)
    dx = # you add code here
    idx2 = # you add code here
    
    diag = # you add code here
    odiag = # you add code here
    oudiag = # you add code here
    oldiag = # you add code here 
    data = np.array(#you add code here)
    dvals = np.array(#you add code here)  
    Amat = spdiags(data, dvals, Nint-1, Nint-1)
    eigenvalues, eigenvectors = eigsh(Amat,10,which='SM',mode='buckling') # return energies and eigenvectors 
   
    return eigenvalues # modify later to find appropriate eigenvectors 

Nvls = 
xvals = np.linspace(-1.,1.,Nvls+1)

tprofile = eval_find(Nvls,xvals,.001)
plt.scatter(np.arange(tprofile.size),tprofile[:])
plt.xlabel("$n$")
plt.ylabel("$E_{n}$")

**Problem 2**: (20 pts) A more realistic way to describe the dynamics of temperature in a narrow corridor is via the _heat equation_, which is a partial differential equation describing how the temperature $T(x,t)$ changes in both space and time.  It is given by

$$
\frac{\partial T}{\partial t} = k \frac{\partial^{2}T}{\partial x^{2}}, ~ a\leq x \leq b
$$

where $k>0$ is the _thermal diffusion_ coefficient, and where we have the _oscillating_ boundary conditions 

$$
\left.\frac{\partial T(x,t)}{\partial x}\right|_{x=a} = \frac{1-\cos(t)}{2}
$$

and

$$
\left.\frac{\partial T(x,t)}{\partial x}\right|_{x=b} = -\sin(t), 
$$

representing two heat flux sources at either end of the domain, such as might arise from the motion of the Sun throughout the day, allowing for cooling on one side and heating on the other, with their roles switching as the Sun moves from East to West.  We likewise have the initial temperature distribution

$$
T(x,0) = u(x).
$$

To numerically solve this, letting $x_{j}= a + j\delta x$, $\delta x = (b-a)/N$

2a) Letting $T_{j}(t)=T(x_{j},t)$, show that the insulating boundary conditions are approximated by the finite-difference approximations:
$$
T_{0}(t) = \frac{2}{3}\left(2T_{1}(t)-\frac{1}{2}T_{2}(t)\right)-\frac{2\delta x}{3} \frac{(1-\cos(t))}{2}, ~ T_{N}(t) = \frac{2}{3}\left(2T_{N-1} - \frac{1}{2}T_{N-2}(t)\right)-\frac{2\delta x}{3} \sin(t).  
$$
(Note, see Homework Ten).  
2b) Using second-order centered-differencing approximations for the $\partial^{2}T/\partial{x^{2}}$ term, show that by discretizing in space, you get the following initial value problem

$$
\frac{d{\bf T}}{dt} = kA{\bf T}, ~ {\bf T}(t) = \begin{pmatrix} T_{1}(t) \\ T_{2}(t) \\ \cdots \\ T_{N-1}(t)\end{pmatrix}, ~ {\bf T}(0) = \begin{pmatrix} u(x_{1}) \\ u(x_{2}) \\ \cdots \\ u(x_{N-1})\end{pmatrix}
$$

where $A$ is some $(N-1)\times (N-1)$ sparse matrix.  What is $A$?  Is $A$ still symmetric?   

2c) Using the Trapezoid Method, we can discretize in time so that if we use time step $\delta t$, we get 

$$
\left(I - \frac{\delta t k}{2} A\right){\bf T}_{m+1} = \left(I + \frac{\delta t k}{2} A\right){\bf T}_{m} + \frac{\delta t k}{2}\left({\bf f}_{m+1} + {\bf f}_{m}\right),
$$

where 

$$
{\bf T}_{m} = {\bf T}(t_{m}) = \begin{pmatrix} T(x_{1},t_{m}) \\ T(x_{2},t_{m}) \\ \cdots \\ T(x_{N-1},t_{m})\end{pmatrix}
$$

and

$$
{\bf f}_{m} = \frac{2}{3\delta x}\begin{pmatrix} -(1-\cos(t_{m}))/2 \\ 0 \\ \vdots \\ 0 \\ -\sin(t_{m}) \end{pmatrix}
$$

Using the code snippet below implement the above scheme to solve the heat equation.  Using the initial heat distribution

$$
u(x) = e^{-(x-5)^{2}}, ~ 0\leq x \leq 10,
$$

generate several plots which show for $0\leq t \leq 10$ how changing the thermal diffusion paramter $k$ from $k=1,10,100$ changes the behavior of the temperature.  Explain your results and comment on any interesting phenomena. 

In [ ]:
def heat_eq_solver(k,u0,Nvls,dx,dt,t0,tf):    
    nsteps = int(np.round((tf-t0)/dt))
    Tsol = np.zeros((Nvls-1,nsteps+1)) # build a matrix to store our solution 
    Tsol[:,0] = u0
    idx2 = 1./(dx**2.)
    diag = -2.*np.ones(Nvls-1)
    udiag = np.ones(Nvls-1)
    ldiag = np.ones(Nvls-1)
    diag[0] = # what goes here?
    diag[Nvls-2] = # what goes here?
    udiag[1] = # what goes here?
    ldiag[Nvls-3] = # what goes here?
    data = np.array([diag,ldiag,udiag])
    dvals = np.array([0,-1,1])
    Amat = idx2*spdiags(data, dvals, Nvls-1, Nvls-1)
    Lp = sid(Nvls-1) + dt*k*Amat/2.
    Lm = sid(Nvls-1) - dt*k*Amat/2.
    for mm in range(0,nsteps):
        # and what about the f vectors?  
        Tsol[:,mm+1] = spsolve(Lm,Lp*Tsol[:,mm]+#something goes here)
    return Tsol

Nvls = int(1e2)
xvals = np.linspace(0.,10.,Nvls+1)
xvalsc = xvals[1:Nvls]
dx = 10./Nvls
dt = 1e-1
t0 = 0.
tf = 10.
k = 1.
NT = int(np.round(tf/dt))

u0 = np.exp(-(xvals[1:Nvls]-5.)**2.)
Tsol = heat_eq_solver(k,u0,Nvls,dx,dt,t0,tf)

plt.subplot(2,1,1)
plt.plot(xvalsc,Tsol[:,0])
plt.xlabel('$x$')
plt.ylabel('$T(x,0)$')

plt.subplot(2,1,2)
plt.plot(xvalsc,Tsol[:,99])
plt.xlabel('$x$')
plt.ylabel('$T(x,t_{f})$')

plt.tight_layout()

**Problem 3** (10pts): A method for solving the matrix problem 

$$
A{\bf x} = {\bf b}
$$

where $A$ is an $n\times n$ matrix, and ${\bf x},{\bf b} \in \mathbb{R}^{n}$, goes as follows.  

* Write $A = D + R$, where $D$ is the diagonal of $A$, and $R= A-D$ is everything not on the diagonal of $A$.
* Given an initial choice of ${\bf x}_{0}$, define the iterative scheme 
$$
{\bf x}_{k+1} = D^{-1}\left({\bf b} - R{\bf x}_{k} \right), ~ k\geq 0.  
$$
* For a user defined tolerance $tol$, stop when 
$$
\left|\left|{\bf x}_{k+1} - {\bf x}_{k} \right|\right|_{2} < tol.  
$$

Given this method then, 

* Complete the code skeleton below.  
* Using the matrices generated by Problem 1, test your code on several different sized matrices, say where $n=10, ~100, ~200$.  Likewise choose ${\bf b}$ to be random vectors of size $10$, $100$, and $200$.  For the $tol$ values of $tol=10^{-4}$ and $tol=10^{-8}$, compare the results of your method to those you get using `spsolve`.  Do you see any variation in the accuracy of your results due to the change in size of the dimension of the problem?    
* For $n=200$, numerically determine the rate of convergence of this method, i.e. find $\alpha$ where
$$
\lim_{k\rightarrow \infty}\frac{\left|\left|{\bf x}_{k+1}-{\bf x}_{\ast}\right|\right|_{2}}{\left|\left|{\bf x}_{k}-{\bf x}_{\ast}\right|\right|^{\alpha}_{2}} = \lambda
$$
and where
$$
A{\bf x}_{\ast} \approx {\bf b},
$$
in other words ${\bf x}_{\ast}$ is the exact solution, or your best approximation to it.  

In [ ]:
def iter_solver(A,b,x0,tol):
    Da = np.diag(A) # returns the diagonal entries of A as an array
    Rmat = # the matrix that is A but with zero diagonal entries
    x1 = (b - np.matmul(Rmat,x0))/Da # I can just divide by an array, 
                                     # because that's all the inverse of a diagonal matrix really asks of us.  
    while np.linalg.norm(x1-x0,2) >= tol: # the norm based tolerance condition, free of charge 
        #write some more stuff
    return #some stuff

btn = np.random.randn(10) # generate a random vector of 10 entries
bhn = np.random.randn(100) 
btwh = np.random.randn(200) 
# and I leave the rest to your capable hands.  
# It's been a great semester.  Have a great break and happy holidays. 